In [ ]:
# SPDX-License-Identifier: Apache-2.0 AND CC-BY-NC-4.0
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<img src="./images/nvmath_head_panel@0.5x.png" alt="nvmath-python" />

# Getting Started with nvmath-python: Direct Sparse Solver

## Exercise: Implement a sparse solver for multiple RHS using nvmath-python

**nvmath-python** supports batched operands in the sparse solver. We distinguish explicit batching, where the samples of a batch are a sequence of matrices (or vectors for the RHS), and implicit batching, where the samples are inferred from 3D or higher-dimensional tensors for the LHS and RHS. The batching for the LHS and RHS can be independent - the LHS can be batched explicitly while the RHS can be batched implicitly and *vice-versa*. Each sample in an explicitly batched system can be of different size (number of equations), resulting in a flexible user interface.

In this exercise you will implement an implicit batching for multiple RHS, represented as a matrix. For simplicity you will use randomly generated RHS. To control the correctness you will compute the L2-norm residual.

### 1. Load matrix object

In [ ]:
import ssgetpy

mtx_obj = ssgetpy.search()[2] # 2 is the index of the matrix in the SuiteSparse Matrix Collection
mtx_obj # Display the matrix object

### 2. Download Matrix Market file as COO matrix

In [ ]:
import scipy

path, _ = mtx_obj.download(extract=True) # Download selected matrix and unpack it
spm_coo = scipy.io.mmread(f"{path}/{path.split('/')[-1]}.mtx") # Read the unpacked Matrix Market file into COO matrix

### 3. Transfer to GPU and run the solver

In [ ]:
import nvmath
import cupy as cp
import cupyx as cpx
from cuda.pathfinder import load_nvidia_dynamic_lib
import os


# Locate the cuDSS library and find the OpenMP multithreading layer
loaded_dl = load_nvidia_dynamic_lib("cudss")
gomp_lib_path = os.path.dirname(loaded_dl.abs_path) + "/libcudss_mtlayer_gomp.so.0"

# Create a cuDSS handle for low-level library access
h = nvmath.bindings.cudss.create()

# Configure solver options with the handle and multithreading library
o = nvmath.sparse.advanced.DirectSolverOptions(
    handle=h,
    multithreading_lib=gomp_lib_path
)


# Transfer sparse matrix to GPU using CuPy
# CuPy can directly convert from SciPy sparse matrices
a = cpx.scipy.sparse.csr_matrix(spm_coo)

# Create the right-hand side vectors for implicit batching. Note that the RHS must be a column-major array
batch_size = 10
b = cp.asfortranarray(cp.random.randn(a.shape[1], batch_size))

# Solve the linear system with GPU-only execution
x = nvmath.sparse.advanced.direct_solver(a, b, options=o)

# Validate the solution
residual = cp.linalg.norm(a @ x - b)
print("Completed successfully!")
print(f"Residual L2 norm ||A*x - b|| = {residual:.2e}")

# Clean up: destroy the cuDSS handle
nvmath.bindings.cudss.destroy(h)